### **Respuestas del examen parcial CC0C2**

#### **Q1 - Tokenización y OOV (2 pts)**

**(a) Conceptos (0.8 pt).**
La **tokenización word-level** asigna **un token por palabra completa**: si una palabra no está en el vocabulario, cae en **OOV** (fuera de vocabulario). En cambio, los métodos de **subpalabras** (por ejemplo, **BPE** o **WordPiece**) **descomponen** las palabras en piezas frecuentes (prefijos, sufijos o segmentos internos), de modo que palabras raras pueden representarse combinando piezas conocidas.

**Cómo reducen OOV.** Si una palabra es desconocida pero sus **subpiezas** sí están en el vocabulario, se evita el token **UNK**: la palabra se expresa como la **secuencia de subpalabras**.

**Beneficio y desventaja.** Beneficio: **mucho menos OOV** y mejor manejo de **morfología rica**. Desventaja: **secuencias más largas** $\Rightarrow$ mayor costo de cómputo (más pasos de atención, mayor latencia).

**(b) Ejercicio guiado (1.2 pts).**
Vocabulario de subpalabras: ${\textbf{öğret},\ \textbf{men},\ \textbf{ler},\ \textbf{imiz},\ \textbf{den}}\ +\ \textbf{UNK}$.
Palabra turca: **öğretmenlerimizden**.
Estrategia: avanzar por **máximo emparejamiento** (*greedy* de izquierda a derecha).

1. **öğret** $\mid$ *menlerimizden*  (coincide "öğret")
2. öğret **men** $\mid$ *lerimizden*
3. öğret men **ler** $\mid$ *imizden*
4. öğret men ler **imiz** $\mid$ *den*
5. öğret men ler imiz **den** $\mid$ *∅*

* **Lista de subpalabras usadas:** $[\text{öğret},\ \text{men},\ \text{ler},\ \text{imiz},\ \text{den}]$
* **Número total de tokens:** $5$
* **¿Hubo UNK?** No.
* **Latencia (frase breve):** **Más tokens $\Rightarrow$ más tiempo por secuencia**, pues crece la longitud efectiva que atraviesa el codificador/atención.


#### **Q2 - Entrenamiento estable y desbalance (2 pts)**

**(a) Clipping sencillo (0.8 pt).**

Dado $g=[4,,3]$ y umbral $\tau=5$, la **norma L2** es:
$$
\lVert g\rVert_2=\sqrt{4^2+3^2}=\sqrt{16+9}=\sqrt{25}=5.
$$

Como $\lVert g\rVert_2=\tau$, **no se clippea** (la regla típica: si $\lVert g\rVert_2>\tau$, se escala).
Si hubiese que escalar, sería
$$
g_{\text{clipped}}=\frac{\tau}{\lVert g\rVert_2},g.
$$

- **Vector final:** $[4,,3]$.
- **Intuición:** el **clipping** limita la magnitud de los gradientes y **evita explosión**, estabilizando pasos de optimización y reduciendo variabilidad en la actualización.

**(b) Desbalance y F1 (1.2 pts).**

Matriz base: $\mathrm{TP}=12,\ \mathrm{FP}=6,\ \mathrm{FN}=9,\ \mathrm{TN}=73$ (umbral $0.5$).

**Precisión** ($P$):
$$
P=\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FP}}=\frac{12}{12+6}=\frac{12}{18}=0.6667.
$$

**Recall** ($R$):
$$
R=\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FN}}=\frac{12}{12+9}=\frac{12}{21}=0.5714.
$$

**F1**:
$$
F1=\frac{2PR}{P+R}=\frac{2\cdot 0.6667\cdot 0.5714}{0.6667+0.5714}\approx 0.6154.
$$

- **Por qué AUC-PR es más informativa en desbalance.** La **PR** se enfoca en **positivos** (precisión y recall), por lo que **refleja mejor la utilidad** cuando los negativos dominan; **ROC** puede verse "optimista" con muchas TN.
- **Mitigación de desbalance:** por ejemplo, **pesos de clase** u **oversampling** de la clase minoritaria.
- **Mejorar calibración:** por ejemplo, **temperature scaling** (ajusta la "temperatura" sobre *logits* para alinear probabilidades con frecuencias reales).


#### **Q3 - Embeddings y similitud (2 pts)**

**(a) Coseno básico (1.0 pt).**

Vectores: $a=[1,0,1],\ b=[0,1,1],\ c=[1,1,0]$.

**Numeradores (productos punto):**
$$
a\cdot b=1\cdot 0+0\cdot 1+1\cdot 1=1,\qquad
a\cdot c=1\cdot 1+0\cdot 1+1\cdot 0=1.
$$

**Normas:**
$$
\lVert a\rVert=\lVert b\rVert=\lVert c\rVert=\sqrt{1^2+0^2+1^2}=\sqrt{2}.
$$

**Similitudes coseno:**
$$
\cos(a,b)=\frac{1}{\sqrt{2}\sqrt{2}}=\frac{1}{2}=0.5,\qquad
\cos(a,c)=\frac{1}{\sqrt{2}\sqrt{2}}=\frac{1}{2}=0.5.
$$

- **Conclusión:** $a$ es **igual de similar** a $b$ y $c$ (ambas $0.5$).
- **Utilidad del coseno:** con vectores **normalizados**, el coseno **mide orientación** (contenido semántico relativo) sin depender de magnitudes.

**(b) Pipeline de oraciones (1.0 pt).**

 - **Modelo léxico:** **GloVe** (preentrenado).
 - **Preprocesamiento mínimo:** minúsculas, quitar signos de puntuación; **stopwords** opcional (según tarea).
 - **Representación de oración:** **promedio** de *embeddings* de las palabras válidas:
$$
\mathbf{s}=\frac{1}{n}\sum_{i=1}^{n}\mathbf{w}_i.
$$

- **Clasificador:** lineal (por ejemplo, regresión logística) con regularización.

- **Mini matriz de confusión 3×3 (ejemplo):**

|       | Pred A | Pred B | Pred C |
| ----- | ------ | ------ | ------ |
| **A** | 7      | 2      | 1      |
| **B** | 1      | 6      | 3      |
| **C** | 0      | 2      | 7      |

Interpretación: **B** se confunde con **C** (por ejemplo, **lematización incompleta** o **entidades compuestas** que el promedio no separa bien).



#### **Q4 - Evaluación y despliegue (2 pts)**

**(a) Métricas y umbral (1.0 pt).**
En binario desbalanceado usaría **$F1$** (balancea precisión/recall) y **AUC-PR** (sensible a la clase positiva).

**Regla simple de umbral operativo:**

* **Opción 1:** elegir el **umbral que maximiza $F1$** en validación.
* **Opción 2 (coste):** fijar umbral que **minimiza**
  $$
  C=c_{\mathrm{FP}}\cdot \mathrm{FP}+c_{\mathrm{FN}}\cdot \mathrm{FN},
  $$
  donde $c_{\mathrm{FP}},c_{\mathrm{FN}}$ reflejan el impacto de errores.

**(b) Reproducibilidad y latencia (1.0 pt).**

**Cuatro prácticas de reproducibilidad:**

1. **Semillas** fijas (Python/NumPy/torch).
2. **Versionado** de datos/código/dependencias (*requirements.lock*).
3. **Preprocesamiento** determinista y documentado.
4. **Artefactos** guardados (modelo, *tokenizer*, *hashes*) para inferencia repetible.

**SLA = $120\ \text{ms}$, p99 actual = $150\ \text{ms}$.**

* **Acción para bajar latencia:** **desactivar ensembling** (un solo modelo), o **cuantizar** a int8, o **acortar secuencia máxima** si no afecta cobertura.
* **Métrica de calidad a vigilar:** por ejemplo, **$F1$** (o **recall@k** si *retrieval*), comparando **antes vs. después** para asegurar que el recorte de latencia **no degrada** el desempeño fuera de tolerancia.


